In [ ]:
print(X_val.shape)
print(X_train.shape, test_no_classes.shape, Y_train.shape)
print(X_train[0], X_train[1])

In [3]:
import numpy as np
import pandas as pd
import random
from IPython.core.display import Image, display

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(13)

num_neuro_1 = 10
num_neuro_2 = 10
num_neuro_3 = 5

RES_DIR = './results/'
DATA_DIR = './CCPP/'
HOME_DIR = './'

In [5]:
train = pd.read_csv(HOME_DIR+'split_data/train_set.csv').values
batch_size = train.shape[0]
in_size = train.shape[1]-1
print(train.shape)

(5158, 5)


In [6]:
class Dense:
    def __init__(self, in_size, out_size):
        np.random.seed(11)
        self.W = np.random.normal(scale=0.01, size=(out_size, in_size))
        self.b = np.random.normal(scale=0.01, size=(out_size))
        
    def forward(self, x):
        self.x = x
        return (self.b + self.W.dot(self.x))
    
    def backward(self, dz, lr=0.001, rgr = 0.01):
        self.dW = np.outer(dz, self.x)
        self.db = dz
        
        self.dx = dz.dot(self.W) 
        
        self.W -= lr*(self.dW + rgr*self.W)
        self.db -= lr*(self.b + rgr*self.b)
        
        return self.dx

In [7]:
class ReLU:
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)
    
    def backward(self, dz):
        dz[self.x < 0] = 0
        return dz

In [8]:
class Softmax:
    def forward(self, x):
        self.x = x
        exps = np.exp(x)
        return exps / np.sum(exps)
    
    def backward(self, dz):
        sm = self.forward(self.x)
        self.lp = (np.eye(sm.shape[0], sm.shape[0]) - sm).T
        self.lp2 = sm * self.lp
        return np.dot(dz, self.lp2)

In [9]:
class Tanh():
    def forward(self, x):
        self.fx = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    
    def backward(self, dz):
        return (1 - self.fx**2)*dz

In [21]:
class Quadratic_loss():
    def __init__(self):
        self.er = 0
        
    def forward(self, y_true, y_pred):
        self.er = y_pred - y_true
        self.n = y_true.shape[0]
        return self.er**2 / self.n
    
    def backward(self, dz):
        return 2/self.n * self.er

In [11]:
class CrossEntropy:
    def forward(self, y_true, y_hat):
        self.y_true = y_true
        self.y_hat = y_hat
        return (-np.sum(y_true*np.log(y_hat)))
    
    def backward(self, dz):
        return (-dz*self.y_true/self.y_hat)

In [12]:
class CifarNet:
    def __init__(self, in_size, out_size):
        self.d1 = Dense(in_size, num_neuro_1)
        self.r1 = Tanh()
        self.d2 = Dense(num_neuro_1, num_neuro_2)
        self.r2 = Tanh()
        self.d3 = Dense (num_neuro_2, num_neuro_3)
        self.r3 = Tanh()
        self.d4 = Dense (num_neuro_3, out_size)
        self.r4 = Tanh()
    
    def forward(self, x):
        y = self.d1.forward(x)
        y = self.r1.forward(y)
        y = self.d2.forward(y)
        y = self.r2.forward(y)
        y = self.d3.forward(y)
        y = self.r3.forward(y)
        y = self.d4.forward(y)
        y = self.r4.forward(y)
        self.y = y
        return y
    
    def backward(self, dz, lr, rr):
        dz = self.r4.backward(dz)
        dz = self.d4.backward(dz, lr, rr)
        dz = self.r3.backward(dz)
        dz = self.d3.backward(dz, lr, rr)
        dz = self.r2.backward(dz)
        dz = self.d2.backward(dz, lr, rr)
        dz = self.r1.backward(dz)
        dz = self.d1.backward(dz, lr, rr)
        return dz

In [13]:
def acc_check(net, validation_X, validation_Y):
    acc = 0.
    for i in range(len(validation_X)):
        y_pred = net.forward(validation_X[i])
        cls_pred = np.argmax(y_pred)
        if(cls_pred == validation_Y[i]):
            acc += 1
    return acc / validation_X.shape[0]

# получение батча для обучения
def get_batch(X, Y, batch_size):
    _, x_batch, _, y_batch = train_test_split(X, Y, test_size = batch_size/X.shape[0])
    return x_batch, y_batch

In [14]:
valid_data = pd.read_csv(HOME_DIR+'split_data/val_set.csv').values
test_data = pd.read_csv(HOME_DIR+'split_data/test_set.csv').values

X_train, Y_train = train[:, 0:in_size], train[:, in_size]
X_val, Y_val = valid_data[:, 0:in_size], valid_data[:, in_size]
X_test, Y_test = test_data[:, 0:in_size], test_data[:, in_size]

In [22]:
net = CifarNet(in_size=in_size, out_size=1)
net

In [23]:

loss = Quadratic_loss()

err_epoh = []
valid_err_epoh = []
test_err_epoh = []
# print(net.d1.W)

In [24]:
# обучаем сеть ... 
learn_rate = 1e1
reg_rate = 0
decay = 0

In [25]:
for epoch in range(100000):
    
    if (epoch%500 == 1):
        plt.grid(True)
        plt.plot(err_epoh, color='r')
        plt.plot(valid_err_epoh, color='b')
        plt.plot(test_err_epoh, color='g')
        
        display.clear_output(wait=True)
        display.display(plt.gcf())
        
        print(epoch)
        print("train: ", err_epoh[epoch-1])
        print("valid: ", valid_err_epoh[epoch-1])
        print("test: ", test_err_epoh[epoch-1])
    
    buffer = 0
    #x, y = get_batch(X_train, Y_train, batch_size)\
    x, y = X_train, Y_train
    for i in range(len(x)):
        y_hat = net.forward(x[i])
        buffer += loss.forward(y[i], y_hat)
        dz = loss.backward(1)
        dz = net.backward(dz, learn_rate, reg_rate)
    buffer /= len(x)
    err_epoh.append(buffer)
    
    
    x, y = X_val, Y_val
    for i in range(len(x)):
        y_hat = net.forward(x[i])
        buffer += loss.forward(y[i], y_hat)
        dz = loss.backward(1)
        dz = net.backward(dz, learn_rate, reg_rate)
    buffer /= len(x)
    valid_err_epoh.append(buffer)
    
    
    x, y = X_test, Y_test
    for i in range(len(x)):
        y_hat = net.forward(x[i])
        buffer += loss.forward(y[i], y_hat)
        dz = loss.backward(1)
        dz = net.backward(dz, learn_rate, reg_rate)
    buffer /= len(x)
    test_err_epoh.append(buffer)
    

plt.plot(loss_list)
plt.show()

IndexError: tuple index out of range

In [42]:
#learn_rate = 1e-3
print(learn_rate)


2.1815008726003495e-05


In [53]:
def make_solution(net, test_no_classes):
    res = []
    for data in test_no_classes:
        predict = np.argmax(net.forward(data))
        res.append(predict)
    return res

In [54]:
solution = make_solution(net, test_no_classes)
with open('my_solution.csv', 'w') as fout:
    print('Id', 'Prediction', sep=',', file=fout)
    for i, prediction in enumerate(solution):
        print(i, prediction, sep=',', file=fout)